<a href="https://colab.research.google.com/github/rocks2021/training-gans/blob/main/gan_even_number_pytorch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

This Colab is coded by Nicolas Bertagnolli. 

In [1]:
from typing import Optional, Tuple, List
import math
import numpy as np
import torch
import torch.nn as nn

In [2]:
def create_binary_list_from_int(number: int) -> List[int]:
    """Creates a list of the binary representation of a positive integer
    Args:
        number: An integer
    Returns:
        The binary representation of the provided positive integer number as a list.
    """
    if number < 0 or type(number) is not int:
        raise ValueError("Only Positive integers are allowed")

    return [int(x) for x in list(bin(number))[2:]]


def generate_even_data(
    max_int: int, batch_size: int = 16
) -> Tuple[List[int], List[List[int]]]:
    """An infinite data generator which yields
    Args:
        max_int: The maximum input integer value
        batch_size: The size of the training batch.
    Returns:
        A Tuple with the labels and the input data.
        labels:
        data:
    """

    # Get the number of binary places needed to represent the maximum number
    max_length = int(math.log(max_int, 2))

    # Sample batch_size number of integers in range 0-max_int
    sampled_integers = np.random.randint(0, int(max_int / 2), batch_size)

    # create a list of labels all ones because all numbers are even
    labels = [1] * batch_size

    # Generate a list of binary numbers for training.
    data = [create_binary_list_from_int(int(x * 2)) for x in sampled_integers]
    data = [([0] * (max_length - len(x))) + x for x in data]

    return labels, data


def convert_float_matrix_to_int_list(
    float_matrix: np.array, threshold: float = 0.5
) -> List[int]:
    """Converts generated output in binary list form to a list of integers
    Args:
        float_matrix: A matrix of values between 0 and 1 which we want to threshold and convert to
            integers
        threshold: The cutoff value for 0 and 1 thresholding.
    Returns:
        A list of integers.
    """
    return [
        int("".join([str(int(y)) for y in x]), 2) for x in float_matrix >= threshold
    ]

In [3]:
class Generator(nn.Module):
    def __init__(self, input_length: int):
        super(Generator, self).__init__()
        self.dense_layer = nn.Linear(int(input_length), int(input_length))
        self.activation = nn.Sigmoid()

    def forward(self, x):
        return self.activation(self.dense_layer(x))


class Discriminator(nn.Module):
    def __init__(self, input_length: int):
        super(Discriminator, self).__init__()
        self.dense = nn.Linear(int(input_length), 1)
        self.activation = nn.Sigmoid()

    def forward(self, x):
        return self.activation(self.dense(x))


class DCGenerator(nn.Module):
    def __init__(self, input_length: int, n_channels: int,  num_base_filters: Optional[int]):
        super(DCGenerator, self).__init__()

        # Calculates the total number of layers
        number_of_layers = int(math.log(self.img_cols, 2) - 3)

        if self.num_base_filters is None:
            num_base_filters = 32 * 2 ** number_of_layers

        # Create the list to hold all sequential layers
        self.layers_list = []

        # Add the initial layer
        self.layers_list.append(nn.Linear(input_length, num_base_filters * 8 * 8))
        self.layers_list.append(nn.ReLU())

        # Add a scaled number of layers
        self.layers_list.append(nn.BatchNorm2d(128))
        self.layers_list.append(nn.Upsample(scale_factor=2))
        self.layers_list.append(nn.Conv2d(128, 128, 3, stride=1, padding=1))
        self.layers_list.append(nn.BatchNorm2d(128, 0.8))
        self.layers_list.append(nn.LeakyReLU(0.2, inplace=True))
        self.layers_list.append(nn.Upsample(scale_factor=2))
        self.layers_list.append(nn.Conv2d(128, 64, 3, stride=1, padding=1))
        self.layers_list.append(nn.BatchNorm2d(64, 0.8))
        self.layers_list.append(nn.LeakyReLU(0.2, inplace=True))
        self.layers_list.append(nn.Conv2d(64, n_channels, 3, stride=1, padding=1))
        self.layers_list.append(nn.Tanh())

        self.layers = nn.ModuleList(self.layers_list)

    def forward(self, x):
        for i, layer in enumerate(self.layers):
            x = layer(x)
        return x


class DCDiscriminator(nn.Module):
    def __init__(self, image_size: int, input_channels: int):
        super(DCDiscriminator, self).__init__()
        self.model = nn.Sequential(
            nn.Conv2d(input_channels, 32, 3, stride=2, padding=1),
            nn.ELU(),
            nn.Dropout2d(0.2),
            nn.Conv2d(32, 64, 3, stride=2, padding=1),
            nn.ELU(),
            nn.BatchNorm2d(64, 0.8),
            nn.Conv2d(64, 128, 3, stride=2, padding=1),
            nn.ELU(),
            nn.BatchNorm2d(128, 0.8),
            nn.Conv2d(128, 256, 3, stride=2, padding=1),
            nn.ELU(),
            nn.Linear(256 * 8 * 8, 1),
            nn.Sigmoid()
        )

    def forward(self, x):
        return self.model(x)

In [4]:
def train(
    max_int: int = 128,
    batch_size: int = 16,
    training_steps: int = 500,
    learning_rate: float = 0.001,
    print_output_every_n_steps: int = 10,
) -> Tuple[nn.Module]:
    """Trains the even GAN
    Args:
        max_int: The maximum integer our dataset goes to.  It is used to set the size of the binary
            lists
        batch_size: The number of examples in a training batch
        training_steps: The number of steps to train on.
        learning_rate: The learning rate for the generator and discriminator
        print_output_every_n_steps: The number of training steps before we print generated output
    Returns:
        generator: The trained generator model
        discriminator: The trained discriminator model
    """
    input_length = int(math.log(max_int, 2))

    # Models
    generator = Generator(input_length)
    discriminator = Discriminator(input_length)

    # Optimizers
    generator_optimizer = torch.optim.Adam(generator.parameters(), lr=learning_rate)
    discriminator_optimizer = torch.optim.Adam(
        discriminator.parameters(), lr=learning_rate
    )

    # loss
    loss = nn.BCELoss()

    for i in range(training_steps):
        # zero the gradients on each iteration
        generator_optimizer.zero_grad()

        # Create noisy input for generator
        # Need float type instead of int
        noise = torch.randint(0, 2, size=(batch_size, input_length)).float()
        generated_data = generator(noise)

        # Generate examples of even real data
        true_labels, true_data = generate_even_data(max_int, batch_size=batch_size)
        true_labels = torch.tensor(true_labels).float()
        true_labels = true_labels.unsqueeze(1)
        true_data = torch.tensor(true_data).float()

        # Train the generator
        # We invert the labels here and don't train the discriminator because we want the generator
        # to make things the discriminator classifies as true.
        generator_discriminator_out = discriminator(generated_data)
        generator_loss = loss(generator_discriminator_out, true_labels)
        generator_loss.backward()
        generator_optimizer.step()

        # Train the discriminator on the true/generated data
        discriminator_optimizer.zero_grad()
        true_discriminator_out = discriminator(true_data)
        true_discriminator_loss = loss(true_discriminator_out, true_labels)

        # add .detach() here think about this
        generator_discriminator_out = discriminator(generated_data.detach())
        # generator_discriminator_loss = loss(
        #     generator_discriminator_out, torch.zeros(batch_size)
        # )
        generator_discriminator_loss = loss(generator_discriminator_out, torch.zeros(batch_size).unsqueeze(1))
        discriminator_loss = (
            true_discriminator_loss + generator_discriminator_loss
        ) / 2
        discriminator_loss.backward()
        discriminator_optimizer.step()
        if i % print_output_every_n_steps == 0:
            print(convert_float_matrix_to_int_list(generated_data))

    return generator, discriminator


if __name__ == "__main__":
    train()

[30, 62, 28, 30, 20, 28, 30, 30, 31, 25, 29, 31, 30, 29, 30, 29]
[29, 30, 62, 30, 30, 30, 28, 29, 29, 63, 21, 28, 62, 30, 30, 63]
[61, 29, 63, 29, 20, 62, 63, 63, 29, 30, 30, 30, 30, 62, 31, 31]
[63, 63, 62, 63, 22, 29, 31, 30, 61, 30, 62, 62, 29, 31, 62, 29]
[59, 30, 31, 29, 63, 30, 63, 30, 63, 63, 31, 29, 62, 63, 58, 63]
[63, 31, 58, 63, 30, 29, 31, 63, 63, 62, 61, 29, 30, 31, 61, 63]
[27, 31, 47, 63, 62, 29, 29, 47, 63, 59, 63, 30, 59, 31, 63, 63]
[63, 30, 59, 47, 29, 30, 22, 63, 63, 31, 62, 59, 63, 31, 59, 31]
[29, 27, 47, 47, 15, 59, 63, 59, 47, 58, 27, 63, 46, 59, 63, 59]
[14, 46, 63, 47, 30, 59, 59, 43, 59, 63, 27, 59, 31, 30, 63, 46]
[63, 59, 43, 46, 59, 27, 47, 47, 59, 59, 47, 59, 25, 47, 47, 43]
[111, 26, 47, 59, 11, 107, 11, 47, 47, 27, 43, 47, 43, 18, 59, 43]
[42, 42, 42, 42, 29, 27, 43, 43, 27, 59, 43, 111, 43, 43, 47, 43]
[43, 43, 27, 59, 43, 47, 43, 27, 42, 107, 107, 42, 107, 27, 127, 43]
[43, 43, 107, 43, 107, 43, 106, 43, 43, 27, 43, 27, 107, 43, 107, 26]
[107, 27, 31,